In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = train.loc[train['fecha_dato']=='2016-05-28']

In [4]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [5]:
from lightfm.data import Dataset

C:\Users\liamc\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [6]:
dataset = Dataset()
dataset.fit(train['ncodpers'],product_cols)

In [7]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 931453, num_items 24.


In [8]:
labels = train[product_cols]

In [9]:
labels['ncodpers'] = train.ncodpers

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
labels = labels.set_index("ncodpers")

In [11]:
stacked_labels = labels.stack()

In [12]:
filtered_labels = stacked_labels.reset_index()

In [13]:
filtered_labels.columns = ["ncodpers", "product", "interaction"]

In [14]:
interactionsdf = filtered_labels.loc[filtered_labels['interaction']==1]

In [15]:
(interactions, weights) = dataset.build_interactions((interactionsdf.iloc[x,0], interactionsdf.iloc[x,1])
                                                     for x in range(len(interactionsdf)))

In [16]:
len(interactionsdf)

1240538

In [17]:
print(repr(interactions))

<931453x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1240538 stored elements in COOrdinate format>


In [18]:
from lightfm import LightFM

model = LightFM(no_components=20,learning_schedule='adagrad',loss='warp')

In [19]:
model.fit(interactions,epochs=40,verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39


In [20]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
users = test.ncodpers.values

In [22]:
users

array([  15889, 1170544, 1170545, ...,  660240,  660243,  660248],
      dtype=int64)

In [25]:
n_users, n_items = dataset.interactions_shape()

In [26]:
item_ids = np.arange(n_items)
model.predict(0,item_ids)

array([-1.29050326, -2.50237751,  2.46573901, -0.72299469,  0.37844777,
       -1.03610599, -0.75957322,  0.69799644,  0.23860197, -0.65274698,
       -0.1703293 , -0.43328977,  0.35205054,  0.03512836, -0.5039621 ,
       -0.00451663, -0.76228404,  0.07885854, -0.88710058,  0.34668446,
        1.26498234,  0.25666806, -0.66574901, -0.59024471])

In [27]:
dataset.mapping()[1][1170544]

924572

In [28]:
preds = []
item_ids = np.arange(n_items)
for user_id in users:
    emb_id = dataset.mapping()[1][user_id]
    scores = model.predict(emb_id,item_ids)
    preds.append(scores)
    scores = []

In [29]:
preds =np.array([np.array(predi) for predi in preds])

In [30]:
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)

In [31]:
preds[0]

array([21,  8,  2, 22, 17, 20, 13, 12, 19, 18, 14, 11,  7,  3,  4, 10, 23,
        9,  6,  0,  5,  1, 15, 16], dtype=int64)

In [32]:
final_month_training_cols = train
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)

In [33]:
#Get the target columns
labels_final_month = train[product_cols]

#Add ncodpers to the dataframe
labels_final_month['ncodpers'] = train.ncodpers

labels_final_month = labels_final_month.set_index("ncodpers")

stacked_labels_final_month = labels_final_month.stack()

filtered_labels_final_month = stacked_labels_final_month.reset_index()

filtered_labels_final_month.columns = ["ncodpers", "product", "newly_added"]

#Only select the rows where there is a new product.
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["newly_added"] == 1]

#Merge with the training features.
multiclass_final_month = filtered_labels_final_month.merge(final_month_training_cols, on="ncodpers", how="left")

final_month = multiclass_final_month.drop_duplicates(multiclass_final_month, keep='last')

labels_final_month_final = final_month['product']

final_month_ncodpers = final_month.ncodpers
#Remove the columns that are not needed to train the model.
final_month = final_month.drop(['ncodpers','newly_added','product'], axis = 1)

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [34]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

In [35]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [36]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])
    if len(used_recommendation_products) % 100000 == 0:
        print(len(used_recommendation_products))

100000
200000
300000
300000
400000
500000
600000
600000
600000
600000
600000
600000


In [37]:
#test_ids are the customer codes for the testing data.
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
target_cols_all = np.array(product_cols)
final_preds = []
#iterate through our model's predictions (pred) and add the 7 most recommended products that the customer does not have.
for idx,predicted in enumerate(preds):
    ids = test_ids[idx]
    top_product = target_cols_all[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))
    if len(final_preds) % 100000 == 0:
        print(len(final_preds))

100000
200000
300000
400000
500000
600000
700000
800000
900000


In [ ]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.shape

In [ ]:
submission.head()